## data

In [10]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans
from random import random
from keras.callbacks import EarlyStopping

Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

# set our random number
matt_rand = round(random() * 3)
    
def adjustData(img,mask,flag_multi_class,num_class):
    
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)


def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
                    mask_color_mode = "rgb",image_save_prefix = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = "D:/Storage/Random_Stuff/Stanl/DigitizationWork/AFSCRunningModel/dev_workflow/example/model/",target_size = (256,256),seed = 1):
    print("batch size: ", batch_size)
    
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    global matt_rand
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in train_generator:
        
        # update our random number (global var)
        matt_rand = round(random() * 3)
        # print(f'for loop, rand={matt_rand}')
        
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)


def testGenerator(test_path,target_size = (256,256),flag_multi_class = False):
    images = glob.glob(test_path + "image*.jpg")
    for img in images:
        img = io.imread(img)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255


def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

## model

In [11]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import tensorflow as tf
from keras.layers import BatchNormalization #add batch normalization



from tensorflow.keras import backend as K
def dice_coef(y_true, y_pred):
#     print("y_true shapes:")
#     print(y_true.shape)
#     print("y_pred shapes:")
#     print(y_pred.shape)
    y_true2 = y_true[:,:,:,0]
    y_true_f = K.flatten(y_true2)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.0001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.0001)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def unet(pretrained_weights = None,input_size = (256,256,3)):
    print("dn") 
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization()(conv1) #added batch normalization after each convolutional layer
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.05), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    # custom loss
#     model.compile(optimizer = Adam(learning_rate = 1e-4), loss = dice_coef_loss, metrics = [dice_coef])
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model    
    #return Model(inputs,conv10)




In [12]:
#from model import *
#from data import *

workspace = "D:/Storage/Random_Stuff/Stanl/DigitizationWork/AFSCRunningModel/dev_workflow/example/samples_reformatted/"

from skimage.transform import rotate

#early_stopping = EarlyStopping(monitor='val_loss', patience=5) #added early stopping

# global variable matt_rand will be updated during transform (inside trainGenerator)
#def matt_process(img):
   # print(f'process: rand={matt_rand}')
 #   if matt_rand == 0:
  #      pass # orig
   # elif matt_rand == 1:
    #    img = rotate(img, 90) # 90deg ccw
    #elif matt_rand == 2:
     #   img = rotate(img, 270) ## 90deg cw
    #else:
     #   img = rotate(img, 180) ## upside-down
    #return img

'''
data_gen_args = dict(preprocessing_function = matt_process,
                    rotation_range=90,
                    width_shift_range=0.01,
                    height_shift_range=0.01,
                    brightness_range=(0.8, 1.2),
                    shear_range=0,
                    zoom_range=0,
                    horizontal_flip=True,
                    fill_mode='nearest')
'''

data_gen_args = dict(rotation_range=10,
                    width_shift_range=0.01,
                    height_shift_range=0.01,
                    brightness_range=(0.8, 1.2),
                    shear_range=0,
                    zoom_range=0,
                    horizontal_flip=True,
                    fill_mode='nearest')

#myGen = trainGenerator(2,workspace,'image','label',data_gen_args,save_to_dir = workspace+'aug')
myGen = trainGenerator(8,workspace,'image','label',data_gen_args,save_to_dir = workspace+'image') #changed from "aug" to "image" because that is the name of the folder

model = unet()

# model_checkpoint = ModelCheckpoint(workspace+'animal_trails_dice_r2000.hdf5', monitor='dice_coef',verbose=1, save_best_only=True)
model_checkpoint = ModelCheckpoint(workspace+'animal_trails_dice_r2000_rotation_loss.hdf5', monitor='loss',verbose=1, save_best_only=True)

model.fit(myGen,steps_per_epoch=21,epochs=10,callbacks=[model_checkpoint]) #5.4375 but rounded down to include all samples bc we have 174 input images/32 batch size


dn
batch size:  8
Found 206 images belonging to 1 classes.
Found 174 images belonging to 1 classes.
Epoch 1/10
 3/21 [===>..........................] - ETA: 20s - loss: 0.7738 - accuracy: 0.6523

ResourceExhaustedError:  OOM when allocating tensor with shape[8,128,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model_3/conv2d_91/Conv2D/Conv2DBackpropInput (defined at C:\Users\Stanl\AppData\Local\Temp\ipykernel_3572\1172610771.py:52) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_15687]

Function call stack:
train_function
